In [ ]:
!pip install keras

In [ ]:
import numpy as np
import pandas as pd
import os
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
#from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

In [ ]:
def csvfile(imagepath, annotationpath):
  imagepath = imagepath
  annotationpath= annotationpath
  df= pd.DataFrame(columns=['image', 'valence','arsousal','expression'])
  for path in os.listdir(imagepath):
    img=path[:-4]
    val=np.load(annotationpath+"/"+path[:-4]+"_val.npy")
    aro=np.load(annotationpath+"/"+path[:-4]+"_aro.npy")
    exp=np.load(annotationpath+"/"+path[:-4]+"_exp.npy")
    df= df.append({"image":img,"valence":val,"arousal":aro,"expression":exp}, ignore_index=True)
  return df

In [ ]:
out= csvfile("/content/image","/content/annotation")

In [ ]:
out

,image,valence,arsousal,expression,arousal
0,112,0.646915,NaN,1,0.0612464
1,65,0.547619,NaN,1,0.00793651
2,207,0,NaN,0,0
3,88,-0.737463,NaN,6,0.468816
4,114,0.0873016,NaN,0,0.34127
...,...,...,...,...,...
139,100,0.00862664,NaN,0,-0.00862664
140,164,0.787009,NaN,1,0.388524
141,29,0.00498107,NaN,0,0.0896593
142,193,0.730159,NaN,1,0.515873


In [ ]:
import cv2
import glob

In [ ]:

def resnet_v1(input_shape, depth, num_classes = 8):
	
	if (depth - 2) % 6 != 0:
		raise ValueError('depth should be 6n + 2 (eg 20, 32, 44 in [a])')
	# Start model definition.
	num_filters = 16
	num_res_blocks = int((depth - 2) / 6)

	inputs = Input(shape = input_shape)
	x = resnet_layer(inputs = inputs)
	# Instantiate the stack of residual units
	for stack in range(3):
		for res_block in range(num_res_blocks):
			strides = 1
			if stack > 0 and res_block == 0: # first layer but not first stack
				strides = 2 # downsample
			y = resnet_layer(inputs = x,
							num_filters = num_filters,
							strides = strides)
			y = resnet_layer(inputs = y,
							num_filters = num_filters,
							activation = None)
			if stack > 0 and res_block == 0: # first layer but not first stack
				# linear projection residual shortcut connection to match
				# changed dims
				x = resnet_layer(inputs = x,
								num_filters = num_filters,
								kernel_size = 1,
								strides = strides,
								activation = None,
								batch_normalization = False)
			x = keras.layers.add([x, y])
			x = Activation('relu')(x)
		num_filters *= 2

	# Add classifier on top.
	# v1 does not use BN after last shortcut connection-ReLU
	x = AveragePooling2D(pool_size = 8)(x)
	y = Flatten()(x)
	outputs = Dense(num_classes,
					activation ='softmax',
					kernel_initializer ='he_normal')(y)

	# Instantiate model.
	model = Model(inputs = inputs, outputs = outputs)
	return model


In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(folder, filename))
            norm = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            if img is not None:
                images.append(img)
            return images

In [ ]:
read_image=load_images_from_folder("/content/image")

In [ ]:
Model=resnet_v1(224*224*, 3)